In [1]:
import os

from PyPDF2 import PdfFileReader, PdfFileWriter
from PyPDF2.generic import DecodedStreamObject, NameObject, DictionaryObject, createStringObject, ArrayObject
from tqdm import tqdm

In [2]:
path = 'Diplomova Praca.pdf'


In [8]:
usb = 'G:/Prilohy/'
attachments_list = os.listdir(usb)
attachments_list = list(map(lambda x: os.path.join(usb,x), attachments_list))

attachments_list = sorted(attachments_list, key=lambda x: str.split(x)[-1])

In [4]:
def appendAttachment(myPdfFileWriterObj, fname, fdata):
    # The entry for the file
    file_entry = DecodedStreamObject()
    file_entry.setData(fdata)
    file_entry.update({NameObject("/Type"): NameObject("/EmbeddedFile")})

    # The Filespec entry
    efEntry = DictionaryObject()
    efEntry.update({ NameObject("/F"):file_entry })

    filespec = DictionaryObject()
    filespec.update({NameObject("/Type"): NameObject("/Filespec"),NameObject("/F"): createStringObject(fname),NameObject("/EF"): efEntry})

    if "/Names" not in myPdfFileWriterObj._root_object.keys():
        # No files attached yet. Create the entry for the root, as it needs a reference to the Filespec
        embeddedFilesNamesDictionary = DictionaryObject()
        embeddedFilesNamesDictionary.update({NameObject("/Names"): ArrayObject([createStringObject(fname), filespec])})

        embeddedFilesDictionary = DictionaryObject()
        embeddedFilesDictionary.update({NameObject("/EmbeddedFiles"): embeddedFilesNamesDictionary})
        myPdfFileWriterObj._root_object.update({NameObject("/Names"): embeddedFilesDictionary})
    else:
        # There are files already attached. Append the new file.
        myPdfFileWriterObj._root_object["/Names"]["/EmbeddedFiles"]["/Names"].append(createStringObject(fname))
        myPdfFileWriterObj._root_object["/Names"]["/EmbeddedFiles"]["/Names"].append(filespec)



In [10]:
fr = PdfFileReader(path)
fw = PdfFileWriter()
fw.appendPagesFromReader(fr)

for my_test in tqdm(attachments_list):
    with open(my_test, 'rb') as my_test_attachment:
        my_test_data = my_test_attachment.read()
    appendAttachment(fw, my_test, my_test_data)

with open('dummy_new.pdf','wb') as file:
    fw.write(file)

100%|██████████| 33/33 [00:45<00:00,  1.38s/it]
